In [4]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
from tensorflow.keras import layers
from os import listdir

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
data_list_model2c = ('/content/drive/MyDrive/Computer Vision/covidDetection/train')
data_list_model2c


'/content/drive/MyDrive/Computer Vision/covidDetection/train'

In [9]:
DATASET_PATH_model2c  = '/content/drive/MyDrive/Computer Vision/covidDetection/train'
train_dir_model2c =  '/content/drive/MyDrive/Computer Vision/covidDetection/train'
test_dir_model2c =  '/content/drive/MyDrive/Computer Vision/covidDetection/test'

In [10]:
IMAGE_SIZE    = (150, 150)
NUM_CLASSES   = len(data_list_model2c)
BATCH_SIZE    = 10  # try reducing batch size or freeze more layers if your GPU runs out of memory
NUM_EPOCHS    = 100
LEARNING_RATE = 0.0001

In [11]:
#Train datagen here is a preprocessor
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=50,
                                   featurewise_center = True,
                                   featurewise_std_normalization = True,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.25,
                                   zoom_range=0.1,
                                   zca_whitening = True,
                                   channel_shift_range = 20,
                                   horizontal_flip = True ,
                                   vertical_flip = True ,
                                   fill_mode='constant' )

/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/image_data_generator.py:342: UserWarning: This ImageDataGenerator specifies `zca_whitening` which overrides setting of`featurewise_std_normalization`.
  warnings.warn('This ImageDataGenerator specifies '


In [12]:
train_batches = train_datagen.flow_from_directory(DATASET_PATH_model2c,
                                                  target_size=IMAGE_SIZE,
                                                  shuffle=True,
                                                  batch_size=BATCH_SIZE,
                                                  subset = "training",
                                                  seed=42,
                                                  class_mode="categorical",)

Found 192 images belonging to 2 classes.


In [13]:
valid_batches = train_datagen.flow_from_directory(DATASET_PATH_model2c,
                                                  target_size=IMAGE_SIZE,
                                                  shuffle=True,
                                                  batch_size=BATCH_SIZE,
                                                  subset = "validation",
                                                  seed=42,
                                                  class_mode="categorical",)

Found 0 images belonging to 2 classes.


In [14]:
vgg16_model = tf.keras.applications.vgg16.VGG16(weights='imagenet', include_top=False, input_tensor=Input(shape=(150,150,3)))

58900480/58889256 [==============================] - 0s 0us/step


In [15]:
for layer in vgg16_model.layers[:-4]:
    layer.trainable = False
model2c = tf.keras.Sequential()
model2c.add(vgg16_model)
model2c.add(Flatten())
model2c.add(Dense(64, activation='relu'))
model2c.add(layers.Dense(2, activation='softmax'))

In [16]:
model2c.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(lr=LEARNING_RATE), metrics=['accuracy'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [17]:
STEP_SIZE_TRAIN=train_batches.n//train_batches.batch_size
STEP_SIZE_VALID=valid_batches.n//valid_batches.batch_size

In [18]:
model2c.fit_generator(train_batches, steps_per_epoch =STEP_SIZE_TRAIN, epochs= 5, verbose=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/image_data_generator.py:720: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/image_data_generator.py:739: UserWarning: This ImageDataGenerator specifies `zca_whitening`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


Epoch 1/5
19/19 [==============================] - 49s 2s/step - loss: 0.5388 - accuracy: 0.7527
Epoch 2/5
19/19 [==============================] - 39s 2s/step - loss: 0.3074 - accuracy: 0.8571
Epoch 3/5
19/19 [==============================] - 40s 2s/step - loss: 0.2961 - accuracy: 0.8791
Epoch 4/5
19/19 [==============================] - 39s 2s/step - loss: 0.1737 - accuracy: 0.9341
Epoch 5/5
19/19 [==============================] - 40s 2s/step - loss: 0.1048 - accuracy: 0.9615


In [19]:
Train_DataGeneration = ImageDataGenerator(rescale=1. / 255)
TrainBatch = Train_DataGeneration.flow_from_directory(train_dir_model2c, target_size=IMAGE_SIZE, batch_size=1, shuffle=False, seed=42, class_mode="categorical")

TrainBatch.reset()
trainResult = model2c.evaluate_generator(TrainBatch, steps = len(TrainBatch), use_multiprocessing = False, verbose = 1, workers=1)
print('Train loss:' , trainResult[0], 'Train accuracy: ', trainResult[1])

prediction = model2c.predict_generator(TrainBatch, steps = len(TrainBatch))
Predicted_class = np.argmax(prediction, axis=1)

print(confusion_matrix(TrainBatch.classes, Predicted_class))

Found 192 images belonging to 2 classes.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  """


192/192 [==============================] - 38s 196ms/step - loss: 0.0784 - accuracy: 0.9740
Train loss: 0.07841301709413528 Train accuracy:  0.9739583134651184


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  


[[88  4]
 [ 1 99]]


In [20]:
## Test accuracy
Test_DataGeneration = ImageDataGenerator(rescale=1. / 255)
TestingBatch = Test_DataGeneration.flow_from_directory(test_dir_model2c, target_size=IMAGE_SIZE, batch_size=1, shuffle=False, seed=42, class_mode="categorical")

TestingBatch.reset()
testResult = model2c.evaluate_generator(TestingBatch, steps = len(TestingBatch), use_multiprocessing = False, verbose = 1, workers=1)
print('Test loss:' , testResult[0], 'Test accuracy: ', testResult[1])

prediction = model2c.predict_generator(TestingBatch, steps = len(TestingBatch))
Predicted_class = np.argmax(prediction, axis=1)

print(confusion_matrix(TestingBatch.classes, Predicted_class))

Found 50 images belonging to 2 classes.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  


50/50 [==============================] - 11s 227ms/step - loss: 0.1356 - accuracy: 0.9200
Test loss: 0.135622039437294 Test accuracy:  0.9200000166893005


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  if __name__ == '__main__':


[[21  4]
 [ 0 25]]
